# Einleitung
Schiffe versenken ist ein simples Spiel, welches von zwei Personen, oder einer Person und einem Computer gespielt werden kann. Ziel des Spiels ist es die von dem gegnerischen Spieler platzierten Schiffe zu versenken. Gewonnen hat der Spieler, welcher als Erstes alle Schiffe des Gegners versenkt hat. Durch den simplen Aufbau ist Schiffe versenken eins der ersten Spiele, die auf einem Computer gespielt werden konnten.

<img src="Images/abbildung2.svg" width="400">  

# Reinforcement Learning
Reinforcement Learning ist eine bewährte Methode, um einem Computersystem beizubringen, ein Spiel zu spielen und in diesem Spiel eigene Strategien zu entwickeln. Diese neu entwickelten Strategien erlauben es, in vielen Fällen, dass das Computersystem einem Menschen in dem Spiel überlegen ist. Ein Agent der jeden Zug zufällig auswählt braucht, bei einem 10x10 Spielfeld mit der klassischen Anzahl von Schiffen, im Durchschnitt ~96 Schüsse. Durch einen Hunt Algorithmus kann die Anzahl der Schüsse auf ~65 Schüsse reduziert werden [1]. Ein durch Reinforcement Learning trainierter Agent kann in der Lage sein ein Spiel in ~52 zu beenden [2]. Damit ist dieser besser als ein Menschlicher Spieler.

Das Hauptkonzept von Reinforcement Learning wird in der Abbildung 1 dargestellt.  
![abbildung1](./images/abbildung1.png)
Figure 1: Reinforcement Learning Prozess  

Ein Agent nimmt durch eine Action Einfluss auf die Umgebung. Im Kontext von Schiffe versenken bedeutet dies, dass ein Feld ausgewählt wird, welches beschossen wird. Die Umgebung gibt dem Agenten dann auf zwei wegen Feedback. Zum Einen gibt die Umgebung Feedback zu dem neuen State der Umgebung also z. B. Treffer, versenkt oder nicht getroffen. Zum Anderen gibt die Umgebung dem Agenten einen Reward. Über diesen Reward bekommt der Agent mitgeteilt, wie gut oder schlecht seine ausgewählte Action war. Ziel des Agent ist es einen möglichst hohen Reward zu bekommen, sprich möglichst Effektiv das Spiel zu gewinnen. Das beschriebene Prinzip wird auch Markov-Decision-Process genannt. Der Markov-Decision-Process nimmt an, dass der aktuelle State einer Umgebung repräsentativ für alle States ist, die vorher in der Umgebung existiert haben.  
	* S: Alle möglichen States
	* A: Alle möglichen Actions
	* R: Reward Verteilung bei aktuellem State und ausgeführter Action (s, a)
	* P: Transition probability. Die Wahrscheinlichkeit welchen State die Umgebung, nach einer ausgeführten Action auf dem aktuellen State, besitzt.
	* ɣ: Discount factor. Hyperparameter, welcher von außen gesetzt werden kann. Beschreibt wie wichtig zukünftige rewards für den aktuellen State sind.

Ziel ist es eine optimale Policy 𝜋* zu finden, die angibt welche Aktion bei einem aktuellen State auszuführen ist.  

$\pi^*$=max($\sum_{t<0} \gamma^t r^t$)

Mathematisch definiert ist diese Policy 𝜋* durch das Maximum aus der Summe aller Rewards. Dabei wird der discount factor benutzt um zukünftige Rewards zu gewichten [1].


# Install Packages

In [ ]:
pip install stable-baselines==2.10.0 tensorflow==1.13.2 gym==0.17.2 numpy

# Umgebung   
Die Umgebung des Spiels Schiffe versenken, kann wie folgt beschrieben werden:  
<img src="Images/tabelle1.png" width="600">  
Die einzige Aktion, die der Agent innerhalb der Umgebung benötigt, ist der Beschuss eines Feldes. Dem Agenten stehen Percepts zur Verfügung. Sie werden über den Zustand jedes einzelnen Feldes zur Verfügung gestellt. Ein Feld kann folgende Informationen enthalten:  
![Tabelle2](./images/tabelle2.png)

# Konfiguration
Die Config Klasse nimmt die Spielkonfiguration entgegen. Die Konfiguration wird von der Umgebung übernommen und ist die Grundlage, auf welcher der Agent das Spiel lernt. Ändern wir Attribute in der Konfiguration, so müssen wir den Agenten auf diese neue Konfiguration und somit neue Umgebung trainieren. 
* board_size: Für das Training ist eine board_size von 5x5 zu empfehlen. Bei 10x10 verlängert sich das Training entsprechend
* ships: Die Schiffe müssen auf das Board passen. Für das Training reichen auch wenige Schiffe
* gap: Zu beachten ist hierbei, dass wenn eine Gap existieren soll, deutlich weniger Platz für die Schiffe zu Verfügung steht
* static_placement: Beim statischen placement ändert sich die Schiffposition nicht bei jedem Durchlauf. Es ist zu erwarten, dass die Agenten bei dieser Einstellung schnell Overfitten
* binary_reward: Gibt an, ob der Agent lernen soll, valide Züge zu spielen, d.h keine Felder doppelt beschießen, oder ob der Agent lernen soll, Schiffe möglichst effektiv zu versenken


In [ ]:
"""
Class representing a config object
"""
class Config:
    """
    Constructor for a Config object
    Arguments:
    board_size = Number of fields in x and y direction.
    ships = Array of Ships with length of ships. Example [2,2,3] Sets 2 Ships with length of 2, 1 ship with length 3
    gap = Boolean: True => Ships have a gap of at least 1 water field between each other.
    False => Ships can be placed side by side
    static_placement => A static placement for ships over all iterations is used
    binary_reward => The reward will be +1 for a valid action and -1 for an invalid action
    to train an agent not to choose the same action multiple times.
    """
    def __init__(self, board_size, ships, gap, static_placement, binary_reward):
        self.board_size = board_size
        self.ships = ships
        self.gap = gap
        self.static_placement = static_placement
        self.binary_reward = binary_reward

# Custom Environment
Für die Umgebung wurde die OpenAI Gym Library verwendet. Diese Library liefert eine Struktur, mithilfe dessen es möglich ist, entweder auf bereits existierende Umgebungen zuzugreifen (https://gym.openai.com/envs/#classic_control) oder eigene Umgebungen zu bauen. Das heißt, für den Anwendungsfall muss ein custom Environment geschrieben werden, welches das Schiffe versenken Spiel implementiert. Folgende Schritte sind dafür notwendig:
1. Die Definition des Environments
2. Löschen eines alten registierten Environments 
3. Registieren des neuen Environments

Die Klasse Ship repräsentiert ein einzelnens Schiff. Es beinhaltet die Startkoordinaten, die Ausrichtung und die Länge des Schiffs. In dieser Klasse wird überprüft, ob ein Schiff getroffen wurde und ob es versunken wurde.

Die Umgebung wird in der BattleshipsEnv Klasse beschrieben. Für die Definition eines eigenen Enviroments mit Hilfe von OpenAI werden folgende Methoden benötigt: 
1. Constructor: initalisiert alle benötigten Variablen.
2. Step: Führt eine Aktion auf der aktuellen Umgebung aus.
3. Reset: Setzt die Umgebung wieder auf den Startzustand.
4. Render: Zeichnet die Umgebung auf der Konsole. Als visuelle Hilfestellung für den Menschen.
5. Close: Schließt die Umgebung.

## Setup
Der Constructor nimmt als Parameter die aktuelle Konfiguration entgegen. Wichitige Variablen sind: 
* self.radar: Das Spielfeld, worauf der Agent seine Actionen ausführt.
* self.avialable_actions: Zu Beginn des Spiel stehen dort alle Koordinatenpaare des Spielfelds, da jedes Koordinatenpaar eine valides Ziel ist. Nach dem Beschuss eines Koordinatenpaares muss dieses Paar aus der Liste der validen Aktionen gestrichen werden. Wenn der Agent wiederholt auf ein Feld schießt, welches nicht mehr in den available_actions vorhanden ist, wird das Spiel sofort beendet, da dies ein fehlerhafter Zug war. 
* self.enemy_board, self.enemyShips: Das Gegnerische Board wird mit Nullen initialsiert. An den Stellen, auf denen ein Schiff liegt wird, eine 1 in das Array geschrieben. Die Schiffe werden durch die Methode place_ships zufällig plaziert.
* self.action_space: Ist abhängig von der Spielfeldgröße. Bei einem Spielfeld 5x5 besitzt der Actionspace eine Größe von 25, bei 10x10 beträgt die Größe 100 
* self.observation_space: Dieser beinhaltet Werte von -1 bis 2, da es 4 Zustände in dem Spiel gibt {'W': 0, 'X': 1, '#': 2, '0': -1}. Diese Werte existieren für jedes Feld.

Die Methode set_up wird zu Beginn jedes Spiels aufgerufen. In dieser Methode werden das Radar und das gegnerische Spielfeld auf deren jeweiligen Startwerte gesetzt. Die avialable_actions werden wieder zurückgesetzt, sodass jeder Zug valide ist, und die Schiffe werden auf dem gegnerischen Spielfeld plaziert. Somit hat man bei jedem Spiel die gleiche Ausgangslage.

## Step
In der Methode werden als erstes die Koordinaten aus dem ausgewählten Action extrahiert. Anschließend wird überprüft, ob die Action valide war. Wenn nicht, wird das Spiel abgebrochen und ein entsprechender negativer Reward vergeben. War die ausgewählte Action valide, so wird das Feld, durch die shoot Methode, beschossen. Nach jedem Schuss wird überprüft, ob das Spiel vorbei ist und der entsprechende Reward vergeben. 

## Reset
Diese Methode wird aufgerufen, wenn ein Spiel vorbei ist. Dabei spielt es keine Rolle, ob das Spiel durch einen invaliden Zug oder durch Gewinnen beendet wurde. In der reset Methode wird die set_up Methode aufgerufen, um ein neues Spiel spielen zu können

## Reward
Der Agent bekommt für jede Action einen Reward. Dabei unterscheidet sich der Reward bei den zwei Trainingvarianten.

Bei dem Training auf valide Spielzüge bekommt der Agent nur einen Reward von +1 wenn er ein Feld beschießt, welches er vorher noch nicht beschossen hatte. Der Agent bekommt einen Reward von -1 wenn er ein Feld beschießt, welches er breits beschossen hat. 

Bei dem Training effektiv das Spiel zu gewinnen unterscheiden sich die Rewards: 
Valider Schuss: 
* Kein Treffer: 0 
* Treffer: +20. Hierbei ist egal, ob der Treffer das Schiff versenkt
* Gewonnen: Reward wird berechnet anhand der benötigten Züge, d.h der Agent bekommt einen höheren Reward, wenn er weniger Züge benötigt: 100 * ((self.board_size*self.board_size) / self.steps) 

Invalider Schuss:
Der Reward wird auf -1000 gesetzt. Diese Zahl ist so hoch gewählt, damit man invalide Schüsse gut erkennen kann, wenn man sich später den Verlauf des Rewards anschaut.



   

In [ ]:
import gym
import numpy as np

from gym.envs.registration import register
from copy import deepcopy
from random import randint, getrandbits
from gym import spaces

"""
Class representing a ship object
"""
class Ship:
    # A ship has zero hits initially
    hits = 0

    """
    Constructor for a Ship object
    Arguments:
    length = Size of the Ship.
    x = Starting X coordinate
    y = Starting Y coordinate
    is_vertical = Boolean whether ship direction is vertically or horizontally
    """
    def __init__(self, length, x, y, is_vertical):
        self.length = length
        self.x = x
        self.y = y
        self.is_vertical = is_vertical

    '''
    Method for counting hits.
    '''
    def hit(self):
        self.hits += 1

    '''
    Method for checking if the ship is sunken.
    '''
    def sunken(self):
        return self.hits == self.length

    '''
    Getter for length of a ship.
    '''
    def get_length(self):
        return self.length

    '''
    Getter for starting x coordinate of a ship.
    '''
    def get_x(self):
        return self.x

    '''
    Getter for starting y coordinate of a ship.
    '''
    def get_y(self):
        return self.y

    '''
    Getter for is_vertical 
    '''
    def get_is_vertical(self):
        return self.is_vertical

    '''
    Method for checking shoot hitting a part of the ship.
    x_hit: x Coordinate of the shoot
    y_hit: y Coordinate of the shoot
    '''
    def is_hit(self, x_hit, y_hit):
        # Iterate all parts of the ship
        for i in range(self.length):
            # Determine placement direction of the ship
            if self.is_vertical:
                # Check if ship part is hit
                if self.x + i == x_hit and self.y == y_hit:
                    # Update hit counter
                    self.hit()
                    return True
            # Ship is placed horizontally
            else:
                # Check if ship part is hit
                if self.x == x_hit and self.y + i == y_hit:
                    # Update hit counter
                    self.hit()
                    return True
        return False



"""
Class representing the Battleship gym environment
"""
class BattleshipsEnv(gym.Env):
  """Custom Environment that follows gym interface"""
  metadata = {'render.modes': ['human']}

  """
  Constructor for the battleships OpenAi gym environment
  Arguments:
  config = Configuration Object for the battleships game.
  """
  def __init__(self, config):
    super(BattleshipsEnv, self).__init__()

    # Instanciate variables
    # Boolean if ships can touch each other
    self.gap = config.gap
    # Boolean if Agent is trained on making valid shoots or playing the game 
    self.binary_reward = config.binary_reward

    # Boolean if ships are placed random each round or stay at fixed position.
    self.static_placement = config.static_placement
    self.placement = None
    self.placement_ships = None

    # The player board "radar" where he registers his shots.
    self.radar = []

    # List containing all valid actions (action get removed after beeing used/shot once)
    self.available_actions = []

    # List containing the ships objects of the enemy
    self.enemyShips = []

    # The enemy board where the ships of the enemy are placed
    self.enemy_board = []

    """
    FieldEncoding to map the state to more human readable content.
    Water:0
    Hit: *
    Sunken: #
    Miss: 0
    """
    self.fieldEncoding = {'W': 0, 'X': 1, '#': 2, '0': -1}

    # Set the available ships to place
    self.ships = config.ships

    # Set the size of the board
    self.board_size = config.board_size

    self.steps = 0

    # Set up the game for a new round
    self.set_up()

    # Allocate possible actions (Each field can be shot => board_size*board_size)
    self.action_space = spaces.Discrete(self.board_size * self.board_size)

    # Allocate oberservations (Each field has a state and can be observed.
    # Possible states are Integer values of fieldEncoding => low=-1, high=2)
    self.observation_space = spaces.Box(low=-1, high=2, shape=(self.board_size, self.board_size),
                                        dtype=np.int)

  '''
  Step function for OpenAi gym.
  Gets called one per round in battleships.
  Arguments:
  action = Integer, the index of the next field to shoot.
  '''
  def step(self, action):
    # Map the action to the board and get x,y coordinates of the next field to shot
    x, y = np.unravel_index(action, (self.board_size, self.board_size))

    # initialize reward with 0
    reward = 0

    #Check if x, y allready have been shot.
    if (x, y) not in self.available_actions:

      double_shot_reward = -1000

      if self.binary_reward:
        double_shot_reward = -1

      return self.radar, double_shot_reward, True, {
        'miss_count': 0,
        'hit_count': 0,
        'empty_count': 0,
        'sunken_count': 0
      }

      # Add negative reward for shooting a forbidden field
      #reward -= 2 * self.board_size

      # Get a random index of available actions
      random_index = np.random.randint(0, len(self.available_actions))

      # Get x,y coordinates from a random valid available action
      #x, y = self.available_actions[random_index]

    # Shoot coordinates
    hit = self.shoot(x, y)

    self.steps += 1

    # Evaluate result of shot
    after_shot_state = self.radar
    miss_after_shot, hit_after_shot, empty_after_shot, sunken_prev_shot = self.count_states(after_shot_state)

    # Add count information to info, for debug and possible calculations of statistics
    info = {
      'miss_count': miss_after_shot,
      'hit_count': hit_after_shot,
      'empty_count': empty_after_shot,
      'sunken_count': sunken_prev_shot
    }

    # Check if game is done
    done = self.check_done()

    # Calculate reward
    reward += self.calculate_reward(hit, done, self.steps)

    if self.binary_reward:
      reward = 1

    return after_shot_state, reward, done, info

  # OpenAI gym reset method. Gets called to set up a new game.
  def reset(self):
    self.set_up()

    # Return the fresh board of the player
    return self.radar

  # OpenAi gym render method
  def render(self, mode='human'):
    for x in range(self.board_size):
      # Print horizontale line
      print("-" * (4 * self.board_size + 2))
      for y in range(self.board_size):
        # Get state of the current field
        current_state_value = self.radar[x, y]
        # Get fieldEncoding for the state of the current field
        current_state = list(self.fieldEncoding.keys())[list(self.fieldEncoding.values()).index(current_state_value)]
        # If state of current Field is Water, print empty space for better readability
        current_state = (current_state if current_state != "W" else " ")
        # Print vertical lines
        print(" | ", end="")
        # Print current state
        print(current_state, end="")
      print(" |")
    print("-" * (4 * self.board_size + 2))

  # OpenAi gym close method
  def close (self):
    print('close')

  # Method to place ships on a given board
  def place_ships(self, board):
    # Initialize variables
    x = 0
    y = 0
    ships = []
    all_ships_placed = False

    # Try to place ships until all ships could be placed
    while not all_ships_placed:
      # Reset placed ships list
      ships = []
      # Reset/Clean the board
      for x in range(self.board_size):
        for y in range(self.board_size):
          board[x, y] = 0
      # No ships have been placed
      ships_placed = 0
      reset = False

      # Iterate over all ships to place
      for ship_length in self.ships:
        # Negative assumption, that the space is occupied
        occupied = True
        trys = 0

        # Try to find a spot to place the ship
        while occupied and not reset:
          # Get new random coordinates on the board
          x, y = self.get_random_coordinates()
          # Get a new random alignment of the ship
          is_vertical = bool(getrandbits(1))
          #Check if the space is allready occupied
          occupied = self.check_occupation(x, y, board, ship_length, is_vertical)
          # Increment the trys for the current ship
          trys += 1
          # If no valid spot can be found, the placement of all ships must be reset
          if trys > 1000:
            reset = True
        # If a valid spot is found:
        if not reset:
          # Create the ship and append it to the ships list
          ships.append(Ship(ship_length, x, y, is_vertical))
          # Increment the number of ships placed
          ships_placed += 1

          # Draw the placed ship onto the board
          if is_vertical:
            for i in range(ship_length):
              board[x + i, y] = 1
          else:
            for i in range(ship_length):
              board[x, y + i] = 1
      # Check if all ships where placed
      if ships_placed == len(self.ships):
        all_ships_placed = True

      if self.static_placement and self.placement is None:
        self.placement = np.copy(board)
        self.placement_ships = deepcopy(ships)

    return ships

  '''
  Method check if the space for a possible ship is occupied on a given boad.
  Arguments:
  x = X Coordinate to place the first part of the ship
  y = Y Coordinate to place the first part of the ship
  board = Board where to check if the ship can be placed on
  ship_length = Length of ship to place
  is_vertical = Boolean if the ship should be placed vertically or horizontally
  '''
  def check_occupation(self, x, y, board, ship_length, is_vertical):
    # Check if ships can touch.
    if self.gap:
      # Check for occupation with a gap
      occupied = self.check_occupation_with_gap(x, y, board, ship_length, is_vertical)
    else:
      # Positive assumption that the space is not occupied
      occupied = False

      # Determine which direction to check
      if is_vertical:
        # Iterate all field of a ship, beginning from the start position (x,y)
        # if the are allready occupied and inside the boundaries of the board
        for i in range(ship_length):
          if x + i >= self.board_size - 1:
            return True
          if board[x + i, y] == 1:
            occupied = True
      else:
        # Same as previous, but horizontally
        for i in range(ship_length):
          if y + i >= self.board_size - 1:
            return True
          if board[x, y + i] == 1:
            occupied = True

    return occupied

  '''
  Method for checking occupation with a gap.
  If a ship is placed vertical, it checks the top row for given coordinate.
  Arguments:
  x = X Coordinate to check on the board
  y = y Coordinate to check on the board
  board = Board to check for occupation
  '''
  def check_occupation_with_gap_top_row_is_vertical(self, x, y, board):
    # If x Coordinate is top row of board, no further checks are needed
    if x == 0:
      return True
    # Check y Coordinate is in bound and left upper field is occupied
    if y != 0 and board[x - 1, y - 1] == 1:
      return True
    # Check center upper field is occupied
    if board[x - 1, y] == 1:
      return True
    # Check y Coordinate is in bound and right upper field is occupied
    if y != self.board_size - 1 and board[x - 1, y + 1] == 1:
      return True

  '''
  Method for checking occupation with a gap.
  If a ship is placed vertical, it checks the left and right column of given coordinate.
  Arguments:
  x = X Coordinate to check on the board
  y = y Coordinate to check on the board
  board = Board to check for occupation
  '''
  def check_occupation_with_gap_left_right_is_vertical(self, x, y, board):
    # Check y Coordinate is in bound and left field is occupied
    if y != 0 and board[x, y - 1] == 1:
      return True
    # Check y Coordinate is in bound and right field is occupied
    if y != self.board_size - 1 and board[x, y + 1] == 1:
      return True

  '''
  Method for checking occupation with a gap.
  If a ship is placed vertical, it checks the bottom row of given coordinate.
  Arguments:
  x = X Coordinate to check on the board
  y = y Coordinate to check on the board
  board = Board to check for occupation
  '''
  def check_occupation_with_gap_bottom_row_is_vertical(self, x, y, board):
    # If x Coordinate is bottom row of board, no further checks are needed
    if x == self.board_size - 1:
      return True
    # Check y Coordinate is in bound and left lower field is occupied
    if y != 0 and board[x + 1, y - 1] == 1:
      return True
    # Check center lower field is occupied
    if board[x + 1, y] == 1:
      return True
    # Check y Coordinate is in bound and right lower field is occupied
    if y != self.board_size - 1 and board[x + 1, y + 1] == 1:
      return True

  '''
  Method for checking occupation with a gap.
  If a ship is placed horizontally, it checks the left column of given coordinate.
  Arguments:
  x = X Coordinate to check on the board
  y = y Coordinate to check on the board
  board = Board to check for occupation
  '''
  def check_occupation_with_gap_left_column(self, x, y, board):
    # If y Coordinate is left column of board, no further checks are needed
    if y == 0:
      return True
    # Check x Coordinate is in bound and left upper field is occupied
    if x != 0 and board[x - 1, y - 1] == 1:
      return True
    # Check center left field is occupied
    if board[x, y - 1] == 1:
      return True
    # Check x Coordinate is in bound and left lower field is occupied
    if x != self.board_size - 1 and board[x + 1, y - 1] == 1:
      return True

  '''
  Method for checking occupation with a gap.
  If a ship is placed horizontally, it checks the top and bottom row of given coordinate.
  Arguments:
  x = X Coordinate to check on the board
  y = y Coordinate to check on the board
  board = Board to check for occupation
  '''
  def check_occupation_with_gap_top_bottom(self, x, y, board):
    # Check x Coordinate is in bound and top field is occupied
    if x != 0 and board[x - 1, y] == 1:
      return True
    # Check x Coordinate is in bound and bottom field is occupied
    if x != self.board_size - 1 and board[x + 1, y] == 1:
      return True

  '''
  Method for checking occupation with a gap.
  If a ship is placed horizontally, it checks the right column of given coordinate.
  Arguments:
  x = X Coordinate to check on the board
  y = y Coordinate to check on the board
  board = Board to check for occupation
  '''
  def check_occupation_with_gap_right_column(self, x, y, board):
    # If y Coordinate is right column of board, no further checks are needed
    if y == self.board_size - 1:
      return True
    # Check x Coordinate is in bound and right upper field is occupied
    if x != 0 and board[x - 1, y + 1] == 1:
      return True
    # Check center right field is occupied
    if board[x, y + 1] == 1:
      return True
    # Check x Coordinate is in bound and right lower field is occupied
    if x != self.board_size - 1 and board[x + 1, y + 1] == 1:
      return True

  '''
  Method for checking occupation with a gap.
  Check whether ship is placed vertically or horizontally. 
  Makes additional checks if coordinates are occupied.
  x = X Coordinate start coordinate to place the ship
  y = y Coordinate start coordinate to place the ship
  board = Board to place the ship on 
  ship_length = Length of the ship to place
  is_vertical = Boolean if the ship should be placed vertically or horizontally
  '''
  def check_occupation_with_gap(self, x, y, board, ship_length, is_vertical):
    # Possitive assumption the field is never occupied
    occupied = False
    # Determine ship placement direction
    if is_vertical:
      # Iterate all field of a ship, beginning from the start position (x,y)
      for i in range(ship_length):
        current_field_x = x + i
        # Check field is inside the boundaries of the board
        if current_field_x >= self.board_size or board[current_field_x, y] == 1:
          occupied = True
          break
        # Check first part of the ship
        if i == 0:
          # Check top row
          if self.check_occupation_with_gap_top_row_is_vertical(current_field_x, y, board):
            occupied = True
            break
          # Check left and right column
          if self.check_occupation_with_gap_left_right_is_vertical(current_field_x, y, board):
            occupied = True
            break
        # Check middle parts of the ship
        elif i < ship_length - 1:
          # Check left and right column
          if self.check_occupation_with_gap_left_right_is_vertical(current_field_x, y, board):
            occupied = True
            break
        # Check last part of the ship
        else:
          # Check bottom row
          if self.check_occupation_with_gap_bottom_row_is_vertical(current_field_x, y, board):
            occupied = True
            break
          # Check left and right column
          if self.check_occupation_with_gap_left_right_is_vertical(current_field_x, y, board):
            occupied = True
            break
    # Ship placement direction is horizontal
    else:
      # Iterate all field of a ship, beginning from the start position (x,y)
      for i in range(ship_length):
        current_field_y = y + i
        # Check field is inside the boundaries of the board
        if current_field_y >= self.board_size or board[x, current_field_y] == 1:
          occupied = True
          break
        # Check first part of the ship
        if i == 0:
          # Check left column
          if self.check_occupation_with_gap_left_column(x, current_field_y, board):
            occupied = True
            break
          # Check top and bottom row
          if self.check_occupation_with_gap_top_bottom(x, current_field_y, board):
            occupied = True
            break
        # Check middle parts of the ship
        elif i < ship_length - 1:
          # Check top and bottom row
          if self.check_occupation_with_gap_top_bottom(x, current_field_y, board):
            occupied = True
            break
        # Check last part of ship
        else:
          # Check right column
          if self.check_occupation_with_gap_right_column(x, current_field_y, board):
            occupied = True
            break
          # Check top and bottom row
          if self.check_occupation_with_gap_top_bottom(x, current_field_y, board):
            occupied = True
            break

    return occupied

  '''
  Method for creating a pair of random coordinates.
  return: Pair of random coordinates
  '''
  def get_random_coordinates(self):
      # random number in range 0 - board_size - 1
      x = randint(0, self.board_size - 1)
      y = randint(0, self.board_size - 1)
      return x, y

  '''
  Method for counting all states currently present on the radar board
  state: Numpy Array of all states present on the radar board
  '''
  def count_states(self, state):
    # Counts how many times a unique element is present on the radar board
    # return the unique element and how many times the unique element is present
    uni_states, counts = np.unique(state.ravel(), return_counts=True)
    hit = counts[uni_states == self.fieldEncoding['X']]
    miss = counts[uni_states == self.fieldEncoding['0']]
    empty = counts[uni_states == self.fieldEncoding['W']]
    sunken = counts[uni_states == self.fieldEncoding['#']]
    # Checks whether hits present and assigns present hits
    if len(hit) == 0:
      hit = 0
    else:
      hit = hit[0]
    # Checks whether misses present and assigns present misses
    if len(miss) == 0:
      miss = 0
    else:
      miss = miss[0]
    # Checks whether empty fields present and assigns present empty fields
    if len(empty) == 0:
      empty = 0
    else:
      empty = empty[0]
    # Checks whether sunken ships present and assigns present sunken ships
    if len(sunken) == 0:
      sunken = 0
    else:
      sunken = sunken[0]

    return miss, hit, empty, sunken

  '''
  Method for shooting on the enemy board.
  Displays the result on radar board
  x: X Coordinate to shoot
  y: Y Coordinate to shoot
  '''
  def shoot(self, x, y):
    # Negative assumption shoot misses a ship
    hit = False
    # Radar board field is set to miss
    self.radar[x, y] = self.fieldEncoding['0']
    # Iterate enemyShips to check for hits
    for ship in self.enemyShips:
      # Check whether shoot is a hit
      if ship.is_hit(x, y):
        # Set radar board field to hit
        self.radar[x, y] = self.fieldEncoding['X']
        hit = True
        # Check whether the ship is sunken
        if ship.sunken():
          # Set radar board ship fields to sunken
          self.draw_sunken(ship, self.radar)

    # Remove shoot Coordinate from list of available actions
    self.available_actions.remove((x, y))
    return hit

  '''
  Method for displaying a sunken ship on radar board.
  ship: Sunken ship
  board: Radar Board 
  '''
  def draw_sunken(self, ship, board):
    # Get start coordinates of the sunken ship
    x = ship.get_x()
    y = ship.get_y()
    # Determine direction of the sunken ship
    if ship.is_vertical:
      # Iterate the sunken ship and update radar board fields to sunken
      for i in range(ship.get_length()):
        board[x + i, y] = self.fieldEncoding['#']
    # sunken ship is placed horizontally
    else:
      # Iterate the sunken ship and update radar board fields to sunken
      for i in range(ship.get_length()):
        board[x, y + i] = self.fieldEncoding['#']

  '''
  Method for checking if the Game is Done.
  All enemy ships must be sunken. 
  '''
  def check_done(self):
    # Possitive assumption the game is allways done
    if self.binary_reward:
      done = False
    else:
      done = True
    #done = False
    # Iterate all enemy ships
    # Check if one of the enemy ships is not yet sunken
    if self.binary_reward:
      if not self.available_actions:
        done = True
    else:
      for ship in self.enemyShips:
        if not ship.sunken():
          done = False
    return done

  '''
  Method for calculating the reward.
  This method must be updated for reward based learning. Currently uses dummy values
  reward: Current Reward of the Agent
  hit: Boolean last shoot was hit or miss 
  done: Boolean game is finished
  '''
  def calculate_reward(self, hit, done, steps):
    # Agent gets a reward for hitting a ship
    reward = 0
    if hit:
      reward += 20
    # Agent gets more reward if he finishes the game
    if done:
     reward += 100 * ((self.board_size*self.board_size) / self.steps)
    # Float for later calculations
    return int(round(reward))

  '''
  Method to setup the game environment.
  '''
  def set_up(self):
    # Inits radar board with Water fields
    self.radar = self.fieldEncoding['W'] * np.ones((self.board_size, self.board_size), dtype='int')
    # Inits enemy board with zeros representing water
    self.enemy_board = 0 * np.ones((self.board_size, self.board_size), dtype='int')
    # resets available_actions
    self.available_actions = []
    # Init available_actions for all fields of the board
    for x in range(self.board_size):
      for y in range(self.board_size):
        self.available_actions.append((x, y))
    # places enemy ships on the enemy board
    if self.placement_ships and self.placement is not None:
      self.enemyShips = deepcopy(self.placement_ships)
      self.enemy_board = np.copy(self.placement)
    else:
      self.enemyShips = self.place_ships(self.enemy_board)

    self.steps = 0

  '''
  Method calculates the maximum mean reward threshold for the callback in training. 
  '''
  def calculate_threshold(self):
    if self.binary_reward:
      return self.board_size * self.board_size
    else:
      ship_fields = 0
      reward = 0
      for ship_length in self.ships:
        ship_fields += ship_length
      reward += ship_fields * 20
      reward += 100 * ((self.board_size*self.board_size) / ship_fields)
      return int(round(reward))


In [ ]:
"""
Class representing a config object
"""
class Config:
    """
    Constructor for a Config object
    Arguments:
    board_size = Number of fields in x and y direction.
    ships = Array of Ships with length of ships. Example [2,2,3] Sets 2 Ships with length of 2, 1 ship with length 3
    gap = Boolean: True => Ships have a gap of at least 1 water field between each other.
    False => Ships can be placed side by side
    static_placement => A static placement for ships over all iterations is used
    binary_reward => The reward will be +1 for a valid action and -1 for an invalid action
    to train an agent not to choose the same action multiple times.
    """
    def __init__(self, board_size, ships, gap, static_placement, binary_reward):
        self.board_size = board_size
        self.ships = ships
        self.gap = gap
        self.static_placement = static_placement
        self.binary_reward = binary_reward
        
"""
Class representing a result object
"""
class Result:
    """
    Constructor for a result object
    Arguments:
    history = List of Tuples with (rounds, action, observation, reward, done, info).
    reward = Overall reward of a played game
    rounds = Amount of rounds used to finish the game
    """
    def __init__(self, history=None, reward=0, rounds=0):
        if history is None:
            history = []
        self.history = history
        self.overall_reward = reward
        self.rounds = rounds
        self.hit_miss_ratio = 0

    '''
    Method for appending a round history to result.
    round: Number of the current round
    action: Performed action
    observation: Radar Board
    reward: Current reward for the performed action
    done: Boolean if game has finished
    info: Debug info
    '''
    def append_history(self, round, action, observation, reward, done, info):
        self.history.append((round, action, observation, reward, done, info))
        self.calculate_overall_reward(reward)

    '''
    Method for setting a the number of rounds to result.
    rounds: Number of rounds used to finish the game
    '''
    def set_rounds(self, rounds):
        self.rounds = rounds

    '''
    Method for calculating the hit/miss ratio.
    hit: Amount of hits
    miss: Amount of misses
    '''
    def calculate_hit_miss_ratio(self, hit, miss):
        if miss is not 0:
            self.hit_miss_ratio = hit / miss

    '''
    Method for calculating the overall reward.
    reward: current reward of the reward
    '''
    def calculate_overall_reward(self, reward):
        self.overall_reward += reward

    '''
    Getter for the overall_reward of result object.
    '''
    def get_overall_reward(self):
        return self.overall_reward

# Welche Probleme entstehen durch einen festen Action-Space?

Die Probleme, die bei einem festen Action-Space entstehen können sind im Beispiel von Schiffe versenken, dass der Agent auf bereits beschossene Felder schießen kann. Die Felder, auf die bereits geschossen worden ist, werden nicht aus dem Action-Space entfernt und der Agent kann so mehrfach auf das gleiche Feld schießen. Dies führt zu verfälschten Ergebnissen und verhindert ein sinvolles Training eines Agenten. Der Action-Space kann in einer OpenAI Gym Umgebung, nicht zur Laufzeit verändert bzw. angepasst werden. Der Agent muss somit selber über das Wissen verfügen, ob eine Aktion valide ist oder nicht. Dieses Verhalten kann dem Agenten antrainiert werden, dies wird im folgedem Abschnitt erläutert. 

# Wie kann ein Agent "valide Züge" erkennen und lernen?
Damit das Problem des festen Action-Spaces umgangen werden kann, muss der Agent lernen,   
nur valide Spielzüge auszuführen. Ein Agent lernt jedoch über eine Maximierung des Rewards.  
Dies führt zu einem Problem, da z. B. ein positiver Reward für das Treffen eines Schiffes vergeben wird.  
Dies Konssequenz ist, dass genau das Gegenteil zum gewünschten Verhalten eintritt. Trifft ein Agent  
ein Schiff, so erhält dieser einen positiven Reward und führt diesen Zug erneut aus, um den positiven Reward  
erneut zu erhalten. Damit dieses Verhalten nicht auftritt, muss ein Agent in 2 Schritten trainiert werden.  
Im ersten Schritt muss der Agent lernen, kein Feld doppelt zu beschießen.
Sobald der Agent dieses Verhalten ausreichend erlernt hat, kann mit diesem "SingleShot"-Modell
das schnellstmögliche Versenken der Schiffe trainiert werden.

# Wie wurde dies im Projekt umgesetzt?
Der Agent muss lernen, zwischen validen und nicht validen Spielzügen zu unterscheiden.
Damit der Agent lernt, nur valide Spielzüge auszuführen, werden folgende Rewards verteilt:  
* -1 Für den Beschuss eines bereits beschossenen Feldes und Abbruch des Spiels.
    ```
      # BattleshipsEnv.py Z:91
      if self.binary_reward:
        double_shot_reward = -1

      return self.radar, double_shot_reward, True, {
        'miss_count': 0,
        'hit_count': 0,
        'empty_count': 0,
        'sunken_count': 0
      }
    ```
* +1 Für den Beschuss eines neuen Feldes.
    ```
    # BattleshipsEnv.py Z:133
    if self.binary_reward:
      reward = 1
    ``` 
Treffer von Schiffen werden hierbei außer Acht gelassen, sodass der höchste Reward erreicht wird, falls alle Felder einmal beschossen werden. Aufgrund des -1 bzw. +1 Reward, wird diese Rewardverteilung im weitern Verlauf als "Binary" bzw. "Binary Reward" bezeichnet. Dieser Binary-Modus kann mithilfe eines Boolean in der Konfiguration festgelegt werden:
    ```
    # TrainACKTR.py Z:12
    # Erstellen der Config und der Environment
    # Der letzte Parameter gibt den Binary-Modus an: True = Binary
    config = Config(5, [3, 2, 2], True, False, True)
    env2 = gym.make('Battleships-v0', config=config)
    ```
Folgend kann der Agent z. B. ACKTR so auf valide Spielzüge trainiert werden.

In [ ]:
import gym_battleships
from stable_baselines.common.policies import MlpPolicy

from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.callbacks import CheckpointCallback, EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines import ACKTR

# Inits Battleship gym environments and config
config = Config(5, [3, 2, 2], True, False, True)
env2 = gym.make('Battleships-v0', config=config)
env3 = gym.make('Battleships-v0', config=config)
env = DummyVecEnv([lambda: env2])
env4 = DummyVecEnv([lambda: env3])


# Define Callback
#Callback stops training if maximum is reached in mean reward
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=env2.calculate_threshold(), verbose=1)
# Callback safes the currently best model
eval_callback = EvalCallback(env4, callback_on_new_best=callback_on_best, verbose=1, best_model_save_path='./ACKTR_Models/best/')
checkpoint_callback = CheckpointCallback(save_freq=1e4, save_path='./model_checkpoints/')


# Uncomment, to train a new fresh model, otherwise a allready trained model will be trained
model = ACKTR(MlpPolicy, env, verbose=2, tensorboard_log="./logs/progress_tensorboard/",  n_cpu_tf_sess=4)

# Train model
# Only 100.000 for demonstration purposes, otherwise use at least 1.000.000
model.learn(100000, callback=[checkpoint_callback, eval_callback])

# Delete current model and load the best model
del model
model = ACKTR.load("./ACKTR_Models/best/best_model.zip", verbose=2, env=env, tensorboard_log="./logs/progress_tensorboard/")



# Test trained model
results = []
for iteration in range(100):
    score = 0
    print('Iteration', iteration)
    # Observed Player board
    observation = env.reset()
    # Init new Result
    result = Result()
    done = False
    # Amount of moves used to finish the game
    rounds = 0
    while not done:
        rounds += 1
        # Get a random action from the action space
        # action = env.action_space.sample()
        # Agent performs a step
        # observation, reward, done, info = env.step(action)
        action, _states = model.predict(observation)
        nextObservation, reward, done, info = env.step(action)
        score += reward
        # Add step to result Object
        result.append_history(rounds, action, nextObservation, reward, done, info)
        observation = nextObservation
        # Game is done
        if done:
            print("End of game: overall_reward=", result.get_overall_reward(), ",rounds", rounds, "score", score)
            # Store amount of rounds in result object
            result.set_rounds(rounds)
            # Add current result object to all results
            results.append(result)
print('Finished')



Mithilfe des [Tensorboards](https://www.tensorflow.org/tensorboard) kann der Fortschritt der Agenten während des Trainings  
beobachtet werden. Um das Tensorboard zu starten, muss folgender Befehl, in der Konsole ausgeführt werden:  
```
tensorboard --port 6004 --logdir ./logs/progress_tensorboard/
```
Die Ausgabe im Tensorboard sieht wie folgt aus:  
![tensorboard](./images/tensorboard.png)  
Hierbei ist zu erkennen, dass sich der "Episode_Reward" dem Wert 25 und somit dem Optimum eines 5x5 Feldes annähert.

Da der Agent nun ausreichend valide Spielzüge beherrscht, kann dieser nun darauf trainiert werden,
mit möglichst wenig Spielzügen bzw. Runden, alle Schiffe zu versenken. Ist der Binary-Modus nicht ausgewählt(False),  
so werden die Rewards auf eine andere Art und Weise verteilt bzw. berechnet:
* -1000 falls ein Feld doppelt beschossen wird, das Spiel wird im Anschluss abgebrochen
    ```
      # BattleshipsEnv.py Z: 89
      # Festlegen des negativ Rewards für doppelten Beschuss
      double_shot_reward = -1000

      # Im Binary Mode wird -1000 mit -1 überschrieben
      if self.binary_reward:
        double_shot_reward = -1
       
      # Reward zurückgeben und Spiel abbrechen
      return self.radar, double_shot_reward, True, {
        'miss_count': 0,
        'hit_count': 0,
        'empty_count': 0,
        'sunken_count': 0
      }
    ```
* Berechnung des Rewards auf der Basis ob ein Schiff getroffen wurde oder nicht
    ```
    # BattleshipsEnv.py Z: 110
    # Shoot coordinates
    hit = self.shoot(x, y)
    
    [...]
    
    # BattleshipsEnv.py Z: 130
    # Reward berechnen
    reward += self.calculate_reward(hit, done, self.steps)
    
    # Falls Binary-Mode wird der Reward mit +1 überschrieben
    if self.binary_reward:
      reward = 1
    
    # Reward, state, info, done zurückgeben
    return after_shot_state, reward, done, info
    ```
    Der Reward wird wie folgt berechnet:
    ```
    '''
    Method for calculating the reward.
    This method must be updated for reward based learning.
    hit: Boolean last shoot was hit or miss 
    done: Boolean game is finished
    '''
    def calculate_reward(self, hit, done):
      # Agent gets a reward for hitting a ship
      reward = 0
      if hit:
        reward += 20
      # Agent gets more reward if he finishes the game
      if done:
       reward += 100 * ((self.board_size*self.board_size) / self.steps)
      return int(round(reward))
    ```
    * +20 Reward für den Treffer eines Schiffs
    * +100 x (25 / Benötigte Rundenanzahl) Bei einem 5x5 Spielfeld, die minimale Rundenanzahl   
    bei einer Belegung mit 3,2,2, ist 7. Dies würde somit einem maximalen Bonus-Reward von   
    100 X (25/7) = 157(gerundet) entsprechen.    
        
Da somit positive Rewards für Treffer von Schiffen vergeben werden, lernt der Agent möglichst schnell, alle Schiffe zu versenken. Das zuvor trainierte Modell kann nun wie folgt weiter trainiert werden:

In [ ]:
from stable_baselines.common.policies import MlpPolicy

from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.callbacks import CheckpointCallback, EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines import ACKTR

# Inits Battleship gym environments and config
config = Config(5, [3, 2, 2], True, False, False)
env2 = gym.make('Battleships-v0', config=config)
env3 = gym.make('Battleships-v0', config=config)
env = DummyVecEnv([lambda: env2])
env4 = DummyVecEnv([lambda: env3])


# Define Callback
#Callback stops training if maximum is reached in mean reward
callback_on_best = StopTrainingOnRewardThreshold(reward_threshold=env2.calculate_threshold(), verbose=1)
# Callback safes the currently best model
eval_callback = EvalCallback(env4, callback_on_new_best=callback_on_best, verbose=1, best_model_save_path='./ACKTR_Models/best/')
checkpoint_callback = CheckpointCallback(save_freq=1e4, save_path='./model_checkpoints/')

# Load current best model
model = ACKTR.load("./ACKTR_Models/best/best_model.zip", verbose=2, env=env, tensorboard_log="./logs/progress_tensorboard/")

# Uncomment the following line to load a pre trained model
#model = ACKTR.load("./ACKTR_Models/ACKTR_5x5_3_2_2_Dynamic.zip", verbose=2, env=env, tensorboard_log="./logs/progress_tensorboard/")

# Train model
# Only 100.000 for demonstration purposes, otherwise use at least 1.000.000
model.learn(100000, callback=[checkpoint_callback, eval_callback])

# Delete current model and load the best model
del model
model = ACKTR.load("./ACKTR_Models/best/best_model.zip", verbose=2, env=env, tensorboard_log="./logs/progress_tensorboard/")



# Test trained model
results = []
for iteration in range(100):
    score = 0
    print('Iteration', iteration)
    # Observed Player board
    observation = env.reset()
    # Init new Result
    result = Result()
    done = False
    # Amount of moves used to finish the game
    rounds = 0
    while not done:
        rounds += 1
        # Get a random action from the action space
        # action = env.action_space.sample()
        # Agent performs a step
        # observation, reward, done, info = env.step(action)
        action, _states = model.predict(observation)
        nextObservation, reward, done, info = env.step(action)
        score += reward
        # Add step to result Object
        result.append_history(rounds, action, nextObservation, reward, done, info)
        observation = nextObservation
        # Game is done
        if done:
            print("End of game: overall_reward=", result.get_overall_reward(), ",rounds", rounds, "score", score)
            # Store amount of rounds in result object
            result.set_rounds(rounds)
            # Add current result object to all results
            results.append(result)
print('Finished')



Mithilfe dieser beiden Trainingsschritte, können die Limitierungen eines festen Action-Spaces von OpenAI Gym 
in einem annehmbaren Maß behoben werden.  
Es wäre jedoch eine Implementation für dynamische Action-Spaces seitens OpenAI Gym sinnvoll, sodass  
Agenten für Spiele wie Schiffe versenken nicht doppelt trainiert werden müssten.  
Dies würde eine Menge an Zeit und somit auch Kosten sparen, welche für das erste Training aufgebracht werden müssen.

# Wie wirken sich unterschiedliche Konfigurationen auf ein Modell aus?
Die Umgebung wurde weitesgehend so programmiert, dass sie über eine Konfigurationsklasse in vielerlei Hinsicht konfigurierbar ist. Dies hat den Vorteil, dass die Agenten theoretisch auf viele verschiedene Gegebenheiten trainiert werden können. Zum Beispiel mit unterschiedlicher Spielfeldgröße, Anzahl der Schiffe oder die Positionierung der Schiffe. Das Problem bei der momentanen Umsetzung ist, dass der Agent durch den festen Action-Space zweimal trainiert werden muss. So wäre es auch, wenn sich die Konfiguration der Umgebung ändert. Daraus resultiert eine große Zeitdauer, zum Trainieren der Agenten. Der ACKTR-Agent schneidet hierbei jedoch besser ab, da der Algorithmus mittels stable-baselines auch multi processed trainiert werden kann und so eine höhere Rechenleistung zur Verfügung steht.

Darüber hinaus hat sich gezeigt, dass die Agenten mit einer Spielfeldgröße von 5x5 gut trainiert werden können. Je größer das Feld ist, desto länger dauert das Training und die 1.000.000 Iterationen reichen nicht aus, um ein erfolgreich trainiertes Modell zu erzielen.

## Quellen
[1] Nick Berry “Battleship” http://www.datagenetics.com/blog/december32011/  
[2] Sue He: 'Deep Reinforcement Learning–of how to win at Battleship' http://www.ccri.com/2017/08/25/deep-reinforcement-learning-win-battleship/  
[3] François-Lavet, Vincent, Peter Henderson, Riashat Islam, Marc G. Bellemare and Joelle Pineau. “An Introduction to Deep Reinforcement Learning.” Found. Trends Mach. Learn. 11 (2018): 219-354.